In [24]:
from helper import *
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri,numpy2ri
from rpy2.robjects.conversion import localconverter
pandas2ri.activate()
import pandas as pd
import os 
import pickle
import numpy as np

In [25]:
ro.r("""source('helper.R')""")

In [26]:


max_id=get_max("Log")
logger_path=os.path.join("Log",f"{max_id}.plot")

exp_id=31
path=f"/home/jianj0c/project/STP/notebook_SPDE/stochasticVolatility/Log/{exp_id}.fitting/fits"
results = []
for file in os.listdir(path):
     fit_path=os.path.join(path,file)
     ro.r(f"""

          xx<-readRDS("{fit_path}")
          result<-parse_result(xx$fit,xx$family)
          """)
     df=ro.r("result$df")
     likelihood=ro.r("unlist(result$like)")
     family={"model":ro.r("xx$family$model")[0],
             "Y":ro.r("xx$family$Y")[0]}
     startValue={"prior.std.dev.nominal":ro.r("xx$start_value$prior.std.dev.nominal")[0],
                    "prior.range.nominal":ro.r("xx$start_value$prior.range.nominal")[0],
                    "rspde.order":ro.r("xx$start_value$rspde.order")[0]}
     
     results.append({"df":ro.conversion.rpy2py(df),
                     "likelihood":likelihood,
                     "family":family,
                     "startValue":startValue,
                     "id":file.split("_")[1][:-4]})

In [27]:
models=dict()
for result in results:
    key=f"{result['family']['model']}_{result['family']['Y']}"
    if key not in models:
        models[key]=list()
    models[key].append(result)

save the parsed result

In [28]:

pickle.dump(models,open(f"/home/jianj0c/project/STP/notebook_SPDE/stochasticVolatility/Log/{exp_id}.fitting/models.pkl","wb"))

In [29]:
models=pickle.load(open(f"/home/jianj0c/project/STP/notebook_SPDE/stochasticVolatility/Log/{exp_id}.fitting/models.pkl","rb"))

In [30]:
models.keys()

dict_keys(['rough_stochvolln', 'rough_stochvol.t', 'rough_stochvol', 'rough_stochvol.nig', 'rs_stochvol'])

In [10]:
start_values_likelihood={}
model_key="rough_stochvol"
models[model_key]=sorted(models[model_key],key=lambda x: f"""({x["startValue"]["prior.std.dev.nominal"]:.0f},{x["startValue"]["prior.range.nominal"]:.0f},{x["startValue"]["rspde.order"]:.0f})""")
for fit in models[model_key]:
    
    statValue=fit["startValue"]
    key=f"""({statValue["prior.std.dev.nominal"]:.0f},{statValue["prior.range.nominal"]:.0f},{statValue["rspde.order"]:.0f})"""
    start_values_likelihood[key]=fit["likelihood"]
    
df=pd.DataFrame(start_values_likelihood)
df.index=df.index+1
df=df.T
df = df.style.format(decimal='.', precision=2)
print(df.to_latex())

\begin{tabular}{lr}
 & 1 \\
(0,115,3) & 3642.93 \\
(0,14,3) & 3642.43 \\
(0,161,4) & 3643.07 \\
(0,196,4) & 3642.46 \\
(0,2,3) & 3642.24 \\
(0,245,4) & 3627.98 \\
(0,250,3) & 3643.22 \\
(1,100,6) & 3641.57 \\
(1,119,5) & 3628.70 \\
(1,158,5) & 3642.72 \\
(1,175,4) & 3642.78 \\
(1,185,5) & 3642.95 \\
(1,189,2) & 3643.33 \\
(1,200,4) & 3642.38 \\
(1,215,6) & 3642.23 \\
(1,217,3) & 3628.60 \\
(1,242,6) & 3641.63 \\
(1,243,3) & 3642.28 \\
(1,244,4) & 3643.21 \\
(1,252,3) & 3642.33 \\
(1,263,6) & 3641.71 \\
(1,269,6) & 3628.15 \\
(1,273,5) & 3642.64 \\
(1,273,6) & 3642.46 \\
(1,275,5) & 3642.35 \\
(1,280,2) & 3643.25 \\
(1,285,2) & 3643.54 \\
(1,285,3) & 3643.27 \\
(1,43,3) & 3643.51 \\
(1,87,4) & 3642.71 \\
\end{tabular}



In [11]:
models.keys()

dict_keys(['rough_stochvolln', 'rough_stochvol.t', 'rough_stochvol', 'rough_stochvol.nig', 'rs_stochvol'])

In [50]:
model_key="rs_stochvol"
max_id_=np.argmax([max(x["likelihood"]) for x in models[model_key] ])
print("config id",models[model_key][max_id_]["id"])
print("run id",np.argmax(models[model_key][max_id_]["likelihood"]))

config id 121
run id 0


In [26]:
models[model_key][max_id_].keys()

dict_keys(['df', 'likelihood', 'family', 'startValue', 'id'])

## gather all models result into one Table

In [41]:
records=[]
for model_key in ["rough_stochvol","rough_stochvol.t","rough_stochvol.nig","rough_stochvolln","rs_stochvol"]:
    if model_key=="rough_stochvol.t":
        max_id_=np.argsort([max(x["likelihood"]) for x in models[model_key] ])[-2]
    else:
        max_id_=np.argmax([max(x["likelihood"]) for x in models[model_key] ])
    fits_for_a_start_values=models[model_key][max_id_]
    

    idd=np.argmax(fits_for_a_start_values["likelihood"])
    df=models[model_key][max_id_]["df"]
    tmp_df=df.loc[[x for x in df.index if x.startswith(f"e{idd+1}")]]
    new_key=model_key.replace("_","-")
    indexx=[f"{new_key}_{x[2:]}" for x in tmp_df.index]
    tmp_df.index=indexx
    records.append(tmp_df)

    

In [42]:
np.argsort(fits_for_a_start_values["likelihood"])

array([0])

In [43]:

df = pd.concat(records).style.format(decimal='.', precision=2)
print(df.to_latex())

\begin{tabular}{lrrrrrr}
 & mean & sd & 0.025quant & 0.5quant & 0.975quant & mode \\
rough-stochvol_std.dev & 0.52 & 0.16 & 0.32 & 0.49 & 0.91 & 0.45 \\
rough-stochvol_range & 207.19 & 105.68 & 80.34 & 180.97 & 482.27 & 141.45 \\
rough-stochvol_nu & 0.36 & 0.07 & 0.25 & 0.35 & 0.51 & 0.34 \\
rough-stochvol.t_std.dev & 0.17 & 0.28 & -0.05 & 0.03 & 0.87 & 0.00 \\
rough-stochvol.t_range & 51.92 & 50.30 & -6.62 & 36.91 & 172.40 & 8.31 \\
rough-stochvol.t_nu & 1.77 & 0.36 & 0.79 & 1.97 & 2.00 & 2.00 \\
rough-stochvol.nig_std.dev & 0.24 & 0.32 & -0.08 & 0.07 & 0.92 & 0.01 \\
rough-stochvol.nig_range & 59.63 & 60.82 & -9.34 & 40.62 & 211.54 & 8.19 \\
rough-stochvol.nig_nu & 1.61 & 0.51 & 0.37 & 1.89 & 2.00 & 2.00 \\
rough-stochvolln_std.dev & 0.46 & 0.13 & 0.28 & 0.43 & 0.78 & 0.40 \\
rough-stochvolln_range & 167.32 & 16.73 & 135.17 & 167.38 & 200.75 & 169.35 \\
rough-stochvolln_nu & 0.25 & 0.06 & 0.14 & 0.25 & 0.40 & 0.24 \\
rs-stochvol_rough_std.dev & 0.32 & 0.37 & 0.03 & 0.18 & 1.51 & 0.10

#### inspect the stability

In [30]:
models["rough_stochvol"][1]

KeyError: 'rough_stochvol'

In [31]:
models.keys()

dict_keys(['rs_stochvol'])

In [39]:
model_key="rs_stochvol"

In [40]:


all=[(x["startValue"],x["df"]["mean"],x["likelihood"]) for x in models[model_key]]
df=pd.concat([x[1] for x in all],axis=1)

df.columns=[f"""({x[0]["rspde.order"]}/{x[0]["prior.range.nominal"]:.4f}/{x[0]["prior.std.dev.nominal"]:.4f})""" for x in all]
df=df.T
df.columns=[x[2:] for x in df.columns]
df["likelihood"]=np.array([x[2] for x in all])

df=df.sort_values("likelihood",ascending=False)
print(df.to_latex(float_format="%.3f"))

\begin{tabular}{lrrrrrrr}
\toprule
 & rough_std.dev & rough_range & rough_nu & smooth_std.dev & smooth_range & smooth_nu & likelihood \\
\midrule
(2/203.0629/0.2727) & 0.318 & 425.381 & 0.276 & 0.559 & 209.159 & 1.021 & 3639.098 \\
(4/280.6559/0.3288) & 0.261 & 753.167 & 0.206 & 0.697 & 116.871 & 1.298 & 3637.651 \\
(3/261.8294/0.1813) & 0.227 & 784.766 & 0.259 & 0.660 & 115.201 & 1.469 & 3637.525 \\
(5/289.6834/0.6559) & 0.181 & 1052.700 & 0.154 & 0.710 & 130.611 & 1.212 & 3637.406 \\
\bottomrule
\end{tabular}

